In [ ]:
import os
import pandas as pd

# iterate over all csv files in test_data/sp500 folder
for file in os.listdir("test_data/sp500"):
    if file.endswith(".csv"):
        # read csv file
        df = pd.read_csv(f"test_data/sp500/{file}", header=[0,1], index_col=0)
        # get ticker symbol from the header
        ticker = df.columns.levels[1][0]
        if df.empty:
            # add ticker to replacement list
            with open("replacement_list.csv", "a") as f:
                f.write(f"{ticker},\n")
            # remove file
            os.remove(f"test_data/sp500/{file}")